In [1]:
import pyodbc
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
db = r'C:\Users\carolirs\Documents\BGCO DB.accdb;'
driver = r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};'
conn = pyodbc.connect(driver + 'DBQ={}'.format(db) )
select_all_query = ''' 
    SELECT rpt_mth
    FROM BGCO_Extract
'''
df_db = pd.read_sql(select_all_query, conn)
                     
conn.close()

In [3]:
df_db.info()
df_db.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245548 entries, 0 to 245547
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   rpt_mth  245548 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 1.9 MB


,rpt_mth
0,2021-06-01
1,2021-06-01
2,2021-06-01
3,2021-06-01
4,2021-06-01


In [4]:
# read latest extract file raw file
fileFolder = r'..\Automation'
fileName = 'latest_bgco_extract.csv'
file = '\\'.join([fileFolder, fileName])
df = pd.read_csv(file)

if df.columns.values[2] == 'CIM_Worker_ID' :
    df = df.rename(columns = {'CIM_Worker_ID' : 'CIM_WORKER_ID'})
    
    
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21607 entries, 0 to 21606
Data columns (total 50 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   rpt_mth                          21607 non-null  object 
 1   stat_date                        21607 non-null  object 
 2   CIM_WORKER_ID                    21607 non-null  int64  
 3   EMPLOYEE_NAME                    21607 non-null  object 
 4   RepHandledCalls                  21607 non-null  int64  
 5   TransferCalls                    21607 non-null  int64  
 6   nps_promoter                     21607 non-null  int64  
 7   nps_detractor                    21607 non-null  int64  
 8   nps_surveys                      21607 non-null  int64  
 9   TechTransfers                    21607 non-null  int64  
 10  Net_OCC                          21607 non-null  float64
 11  disconnects                      21607 non-null  int64  
 12  RepeatCalls2hr    

,rpt_mth,stat_date,CIM_WORKER_ID,EMPLOYEE_NAME,RepHandledCalls,TransferCalls,nps_promoter,nps_detractor,nps_surveys,TechTransfers,...,CLNR_Per_Call_Numerator,CLNR_Per_Call_Denominator,CLNR_NTF_Numerator,CLNR_NTF_Denominator,Remedy_Return_Numerator,Remedy_Return_Denominator,TRG_Util_Numerator,TRG_Util_Denominator,composite_revenue_numerator,composite_revenue_denominator
0,2022-02-01,2022-02-11,7299742,"SARMIENTO, QUIAN",28,3,0,0,0,3,...,1,28,0,0,0,0,7,28,0.0,28
1,2022-02-01,2022-02-11,7299753,"BULDA, ROSAL BERNADETT",29,2,2,1,3,0,...,0,29,0,0,0,0,0,29,0.0,29
2,2022-02-01,2022-02-11,7299777,"MAGSIPOC, SOFIA",21,2,0,1,1,1,...,0,21,0,0,0,0,1,21,0.0,21
3,2022-02-01,2022-02-11,7299780,"MANAOG, VERNON",21,4,1,0,1,0,...,1,21,0,0,0,0,0,21,0.0,21
4,2022-02-01,2022-02-11,7299784,"OBERA, KAREEN CASEY",27,5,0,0,1,1,...,0,27,0,0,0,0,5,27,0.0,27


In [5]:
# clean datetime cols
currentMonth = str(datetime.now().month)
currentYear = str(datetime.now().year)
curr_rpt_mth = currentYear + '/' + currentMonth + '/' + str(1)
curr_rpt_mth = pd.to_datetime(curr_rpt_mth)

prevMonth = str(datetime.now().month - 1)
prevYear = str(datetime.now().year)
prev_rpt_mth = prevYear + '/' + prevMonth + '/' + str(1)
prev_rpt_mth = pd.to_datetime(prev_rpt_mth)

prev2Month = str(datetime.now().month - 2)
prev2Year = str(datetime.now().year)
prev2_rpt_mth = prev2Year + '/' + prev2Month + '/' + str(1)
prev2_rpt_mth = pd.to_datetime(prev2_rpt_mth)

dt_cols = ['rpt_mth', 'stat_date']
for col in dt_cols :
    df[col] = pd.to_datetime(df[col]).dt.normalize()
    
print(curr_rpt_mth)
print(prev_rpt_mth)
print(prev2_rpt_mth)

2022-03-01 00:00:00
2022-02-01 00:00:00
2022-01-01 00:00:00


In [6]:
# assign vals and count per mth vals
df_curr_mth = df.loc[df['rpt_mth'] == curr_rpt_mth]
df_prev_mth = df.loc[df['rpt_mth'] == prev_rpt_mth]
df_prev2_mth = df.loc[df['rpt_mth'] == prev2_rpt_mth]

df_db['rpt_mth'].value_counts().sort_index(ascending = False)

2022-03-01     7898
2022-02-01    13042
2022-01-01    12846
2021-12-01    16745
2021-11-01    16500
2021-10-01    15753
2021-09-01    16001
2021-08-01    16709
2021-07-01    17605
2021-06-01    17543
2021-05-01    18480
2021-04-01    19776
2021-03-01    20454
2021-02-01    16590
2021-01-01    19606
Name: rpt_mth, dtype: int64

In [7]:
# bool check
if curr_rpt_mth in df['rpt_mth'].values :
    df_db.drop(df_db.loc[df_db.rpt_mth == curr_rpt_mth].index, inplace = True)
    curr_rpt_mth_bool = True
else :
    curr_rpt_mth_bool = False

if prev_rpt_mth in df['rpt_mth'].values :
    df_db.drop(df_db.loc[df_db.rpt_mth == prev_rpt_mth].index, inplace = True)
    prev_rpt_mth_bool = True
else :  
    prev_rpt_mth_bool = False
    
if prev2_rpt_mth in df['rpt_mth'].values :
    df_db.drop(df_db.loc[df_db.rpt_mth == prev2_rpt_mth].index, inplace = True)
    prev2_rpt_mth_bool = True
else :  
    prev2_rpt_mth_bool = False
    
print('Total Rows: ', len(df_db))
df_db['rpt_mth'].value_counts().sort_index(ascending = False)

Total Rows:  224608


2022-01-01    12846
2021-12-01    16745
2021-11-01    16500
2021-10-01    15753
2021-09-01    16001
2021-08-01    16709
2021-07-01    17605
2021-06-01    17543
2021-05-01    18480
2021-04-01    19776
2021-03-01    20454
2021-02-01    16590
2021-01-01    19606
Name: rpt_mth, dtype: int64

In [8]:
# concat dfs and clean cols
frames = [df_curr_mth, df_prev_mth, df_prev2_mth]

df_updated = pd.concat(frames, axis = 0, join = 'outer')

na_cols = ['rep_nps_promoter', 'rep_nps_detractor', 'rep_nps_surveys', 'Total_Upgrades', 
           'rep_For_Me_Yes', 'rep_For_Me_Surveys', 'rep_Reliably_Yes', 'rep_Reliably_Surveys'
          ]
for na_col in df_updated[na_cols] :
    df_updated[na_col].fillna(0, inplace = True)
    
all_cols = ['rpt_mth', 'stat_date', 'CIM_WORKER_ID', 'EMPLOYEE_NAME',
       'RepHandledCalls', 'TransferCalls', 'nps_promoter', 'nps_detractor',
       'nps_surveys', 'TechTransfers', 'Net_OCC', 'disconnects',
       'RepeatCalls2hr', 'TotalCallsFor2hrRepeats', 'RepeatCalls3Day',
       'TotalCallsFor3DayRepeats', 'Accessory_Amt', 'rep_nps_promoter',
       'rep_nps_detractor', 'rep_nps_surveys', 'Total_Upgrades',
       'rep_For_Me_Yes', 'rep_For_Me_Surveys', 'rep_Reliably_Yes',
       'rep_Reliably_Surveys', 'Available_Time', 'Signed_On_Time', 'Talk_Time',
       'Hold_Time', 'Work_Time', 'VIRTUAL_LOCATION_DESCRIPTION',
       'Phone_VZ_Satisfaction_Completed', 'Phone_VZ_Satisfaction_Surveys',
       'Phone_Overall_Int_Completed', 'Phone_Overall_Int_Surveys',
       'acd_calls_handled', 'Repeat_1Day_Numerator', 'Repeat_1Day_Denominator',
       'MVD_Util_Numerator', 'MVD_Util_Denominator', 'CLNR_Per_Call_Numerator',
       'CLNR_Per_Call_Denominator', 'CLNR_NTF_Numerator',
       'CLNR_NTF_Denominator', 'Remedy_Return_Numerator',
       'Remedy_Return_Denominator', 'TRG_Util_Numerator',
       'TRG_Util_Denominator', 'composite_revenue_numerator',
       'composite_revenue_denominator']

for c in all_cols :
    if type(df_updated[c][1]) == np.int64 :
        df_updated[c] = df_updated[c].astype(float)
    
df_updated.info()
df_updated.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21607 entries, 40 to 21606
Data columns (total 50 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   rpt_mth                          21607 non-null  datetime64[ns]
 1   stat_date                        21607 non-null  datetime64[ns]
 2   CIM_WORKER_ID                    21607 non-null  float64       
 3   EMPLOYEE_NAME                    21607 non-null  object        
 4   RepHandledCalls                  21607 non-null  float64       
 5   TransferCalls                    21607 non-null  float64       
 6   nps_promoter                     21607 non-null  float64       
 7   nps_detractor                    21607 non-null  float64       
 8   nps_surveys                      21607 non-null  float64       
 9   TechTransfers                    21607 non-null  float64       
 10  Net_OCC                          21607 non-null  float64 

,rpt_mth,stat_date,CIM_WORKER_ID,EMPLOYEE_NAME,RepHandledCalls,TransferCalls,nps_promoter,nps_detractor,nps_surveys,TechTransfers,...,CLNR_Per_Call_Numerator,CLNR_Per_Call_Denominator,CLNR_NTF_Numerator,CLNR_NTF_Denominator,Remedy_Return_Numerator,Remedy_Return_Denominator,TRG_Util_Numerator,TRG_Util_Denominator,composite_revenue_numerator,composite_revenue_denominator
40,2022-03-01,2022-03-07,8909745.0,"MASAGCA JR., THOMAS",28.0,2.0,1.0,0.0,1.0,1.0,...,0.0,28.0,0.0,0.0,0.0,0.0,2.0,28.0,0.0,28.0
41,2022-03-01,2022-03-07,8909746.0,"MAYODONG, AIVEE FLOR",30.0,4.0,0.0,0.0,0.0,0.0,...,0.0,30.0,0.0,0.0,0.0,0.0,1.0,30.0,0.0,30.0
42,2022-03-01,2022-03-07,8909747.0,"NEDIA, KIM",28.0,4.0,3.0,0.0,3.0,0.0,...,0.0,28.0,0.0,0.0,0.0,0.0,0.0,28.0,0.0,28.0
43,2022-03-01,2022-03-07,8909752.0,"SELVA, JOHN WINGIL",29.0,5.0,2.0,0.0,3.0,0.0,...,0.0,29.0,0.0,0.0,0.0,1.0,2.0,29.0,0.0,29.0
44,2022-03-01,2022-03-07,8909760.0,"DELA CRUZ, JENNA CARLA",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# save file for backup or checking
fileFolder = r'..\Automation'
fileName = 'clean_latest_bgco_extract_frm_python.csv'
file = '\\'.join([fileFolder, fileName])
df_updated.to_csv(file)

In [10]:
# MS Access SQL query
# delete data based on if there is an update on rpt_mth in BGCO_Extract

db = r'..\Documents\BGCO DB.accdb;'
driver = r'Driver = {Microsoft Access Driver (*.mdb, *.accdb)};'
conn = pyodbc.connect(driver + 'DBQ={}'.format(db) )
cursor = conn.cursor()

if curr_rpt_mth_bool == True :
    del_curr_rpt_mth_query = '''
        DELETE 
        FROM BGCO_Extract
        WHERE rpt_mth = #{}#
    '''.format(curr_rpt_mth)
    conn.execute(del_curr_rpt_mth_query)

if prev_rpt_mth_bool == True :
    del_prev_rpt_mth_query = '''
        DELETE 
        FROM BGCO_Extract
        WHERE rpt_mth = #{}#
    '''.format(prev_rpt_mth)
    conn.execute(del_prev_rpt_mth_query)
    
if prev2_rpt_mth_bool == True :
    del_prev2_rpt_mth_query = '''
        DELETE 
        FROM BGCO_Extract
        WHERE rpt_mth = #{}#
    '''.format(prev2_rpt_mth)
    conn.execute(del_prev2_rpt_mth_query)
    
cursor.commit() 

In [11]:
# insert clean df_updated to BGCO_Extract table
insert_df = '''
    INSERT INTO 
        BGCO_Extract(
            rpt_mth, 
            stat_date, 
            CIM_WORKER_ID, 
            EMPLOYEE_NAME, 
            RepHandledCalls, 
            TransferCalls, 
            nps_promoter, 
            nps_detractor, 
            nps_surveys, 
            TechTransfers, 
            Net_OCC, 
            disconnects, 
            RepeatCalls2hr, 
            TotalCallsFor2hrRepeats, 
            RepeatCalls3Day, 
            TotalCallsFor3DayRepeats, 
            Accessory_Amt, 
            rep_nps_promoter, 
            rep_nps_detractor, 
            rep_nps_surveys, 
            Total_Upgrades, 
            rep_For_Me_Yes, 
            rep_For_Me_Surveys, 
            rep_Reliably_Yes, 
            rep_Reliably_Surveys, 
            Available_Time, 
            Signed_On_Time, 
            Talk_Time, 
            Hold_Time, 
            Work_Time, 
            VIRTUAL_LOCATION_DESCRIPTION, 
            Phone_VZ_Satisfaction_Completed, 
            Phone_VZ_Satisfaction_Surveys, 
            Phone_Overall_Int_Completed, 
            Phone_Overall_Int_Surveys, 
            acd_calls_handled, 
            Repeat_1Day_Numerator, 
            Repeat_1Day_Denominator, 
            MVD_Util_Numerator, 
            MVD_Util_Denominator, 
            CLNR_Per_Call_Numerator, 
            CLNR_Per_Call_Denominator, 
            CLNR_NTF_Numerator, 
            CLNR_NTF_Denominator, 
            Remedy_Return_Numerator, 
            Remedy_Return_Denominator, 
            TRG_Util_Numerator, 
            TRG_Util_Denominator, 
            composite_revenue_numerator, 
            composite_revenue_denominator  
            
            ) 
        
        VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, 
                ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, 
                ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, 
                ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, 
                ?, ?, ?, ?, ?, ?, ?, ?, ?, ? 
            )

'''

for idx, row in df_updated.iterrows(): 
    cursor.execute(insert_df, 
                   row['rpt_mth'], 
                   row['stat_date'], 
                   row['CIM_WORKER_ID'],
                   row['EMPLOYEE_NAME'],
                   row['RepHandledCalls'],
                   row['TransferCalls'],
                   row['nps_promoter'],
                   row['nps_detractor'],
                   row['nps_surveys'],
                   row['TechTransfers'],
                   row['Net_OCC'],
                   row['disconnects'],
                   row['RepeatCalls2hr'],
                   row['TotalCallsFor2hrRepeats'],
                   row['RepeatCalls3Day'],
                   row['TotalCallsFor3DayRepeats'],
                   row['Accessory_Amt'],
                   row['rep_nps_promoter'],
                   row['rep_nps_detractor'],
                   row['rep_nps_surveys'],
                   row['Total_Upgrades'],
                   row['rep_For_Me_Yes'],
                   row['rep_For_Me_Surveys'],
                   row['rep_Reliably_Yes'],
                   row['rep_Reliably_Surveys'],
                   row['Available_Time'],
                   row['Signed_On_Time'],
                   row['Talk_Time'],
                   row['Hold_Time'],
                   row['Work_Time'],
                   row['VIRTUAL_LOCATION_DESCRIPTION'],
                   row['Phone_VZ_Satisfaction_Completed'],
                   row['Phone_VZ_Satisfaction_Surveys'],
                   row['Phone_Overall_Int_Completed'],
                   row['Phone_Overall_Int_Surveys'],
                   row['acd_calls_handled'],
                   row['Repeat_1Day_Numerator'],
                   row['Repeat_1Day_Denominator'],
                   row['MVD_Util_Numerator'],
                   row['MVD_Util_Denominator'],
                   row['CLNR_Per_Call_Numerator'],
                   row['CLNR_Per_Call_Denominator'],
                   row['CLNR_NTF_Numerator'],
                   row['CLNR_NTF_Denominator'],
                   row['Remedy_Return_Numerator'],
                   row['Remedy_Return_Denominator'],
                   row['TRG_Util_Numerator'],
                   row['TRG_Util_Denominator'],
                   row['composite_revenue_numerator'],
                   row['composite_revenue_denominator']
                   
                  ) 

cursor.commit()

In [12]:
select_all_query = ''' 
    SELECT *
    FROM BGCO_Extract
    ORDER BY stat_date DESC
'''    
   
df_db_updated = pd.read_sql(select_all_query, conn)

print(df_db_updated['rpt_mth'].value_counts().sort_index(ascending = False) )
df_db_updated.info()
df_db_updated.head()
conn.close()

del df_db_updated
del df_updated
del df_db


2022-03-01     8517
2022-02-01    13090
2022-01-01    12846
2021-12-01    16745
2021-11-01    16500
2021-10-01    15753
2021-09-01    16001
2021-08-01    16709
2021-07-01    17605
2021-06-01    17543
2021-05-01    18480
2021-04-01    19776
2021-03-01    20454
2021-02-01    16590
2021-01-01    19606
Name: rpt_mth, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246215 entries, 0 to 246214
Data columns (total 50 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   rpt_mth                          246215 non-null  datetime64[ns]
 1   stat_date                        246215 non-null  datetime64[ns]
 2   CIM_WORKER_ID                    246215 non-null  float64       
 3   EMPLOYEE_NAME                    246215 non-null  object        
 4   RepHandledCalls                  246215 non-null  float64       
 5   TransferCalls                    246215 non-null  float64       
 

In [13]:
# automate refresh of piv tables

import win32com.client as win32

xl = win32.DispatchEx('Excel.Application')
xl.DisplayAlerts = False
xl.Visible = True

xl_file = xl.Worbooks.Open('..\PB\BGCO Playbook 2022.xlsb')

# Refresh All Tables
xl_file.RefreshAll()
xl_file.Save()
xl_file.Close()

xl.Quit()

del xl_file
del xl

In [14]:
import winsound
import time

def sound():
  duration = 500  # milliseconds
  freq = 660  # Hz
  winsound.Beep(freq, duration)
  winsound.Beep(freq, duration)
  time.sleep(0.5)
  winsound.Beep(freq, duration)


print('SFTP BGCO Extract update complete!')
sound()

SFTP BGCO Extract update complete!
